<a href="https://colab.research.google.com/github/PremGorecki/NeuralNetwork/blob/main/03_keras/04_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Update: Należy użyć biblioteki tensorflow w wersji 2.0
!pip install -q tensorflow==2.0

In [ ]:
# %tensorflow_version 2.x  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

np.set_printoptions(precision=12, suppress=True, linewidth=120)
print(tf.__version__)

Załadowanie i przygotowanie danych

In [ ]:
(X_train, y_train), (X_test, y_test) = load_data()

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
print(X_train[0])

In [ ]:
print(f'X_train[0] shape: {X_train[0].shape}')

In [7]:
X_train = X_train / 255.
X_test = X_test / 255.

In [ ]:
#Eksploracja danych

plt.imshow(X_train[0], cmap='gray_r')
plt.axis('off')

In [ ]:
plt.figure(figsize=(13, 13))
for i in range(1, 11):
    plt.subplot(1, 10, i)
    plt.axis('off')
    plt.imshow(X_train[i-1], cmap='gray_r')
    plt.title(y_train[i-1], color='black', fontsize=16)
plt.show()

Budowa sieci neuronowej

In [ ]:
def build_model():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=10, activation='softmax'))

    model.compile(optimizers='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

model = build_model()
model.summary()

Trenowanie modelu

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

ModelCheckpoint - Zapisywanie wag po każdej poprawie modelu
Za każdym razem gdy monitorowana metryka ulega poprawie wagi są zapisywane do pliku.
Wykorzystamy klasę ModelCheckpoint do obserwowania metryki accuracy na zbiorze walidacyjnym.

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint

!mkdir model
filepath ='model/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
#polecenie bash'owe do wyświetlenia katalogu w którym się znajdujemy
!ls

In [ ]:
model = build_model()
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, callbacks=[checkpoint])

ModelCheckpoint - Zapisanie najlepszego modelu

In [20]:
filepath = 'best_model_weights.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model = build_model()
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, callbacks=[checkpoint])

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
47648/48000 [============================>.] - ETA: 0s - loss: 0.3265 - accuracy: 0.9056
Epoch 00001: val_accuracy improved from -inf to 0.95100, saving model to best_model_weights.hdf5
48000/48000 [==============================] - 6s 128us/sample - loss: 0.3256 - accuracy: 0.9058 - val_loss: 0.1658 - val_accuracy: 0.9510
Epoch 2/10
47968/48000 [============================>.] - ETA: 0s - loss: 0.1662 - accuracy: 0.9521
Epoch 00002: val_accuracy improved from 0.95100 to 0.96175, saving model to best_model_weights.hdf5
48000/48000 [==============================] - 6s 119us/sample - loss: 0.1663 - accuracy: 0.9520 - val_loss: 0.1325 - val_accuracy: 0.9617
Epoch 3/10
47840/48000 [============================>.] - ETA: 0s - loss: 0.1338 - accuracy: 0.9622
Epoch 00003: val_accuracy improved from 0.96175 to 0.96967, saving model to best_model_weights.hdf5
48000/48000 [==============================] - 6s 117us/sample - loss: 0.13

Załadowanie wag i ocena modelu

In [ ]:
best_model = Sequential()
best_model.add(Flatten(input_shape=(28, 28)))
best_model.add(Dense(units=128, activation='relu'))
best_model.add(Dropout(0.2))
best_model.add(Dense(units=10, activation='softmax'))

best_model.compile(optimizers='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

filepath = 'best_model_weights.hdf5'
best_model.load_weights(filepath)

#test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
#print(test_accuracy)

Early Stopping - wczesne zatrzymanie
Klasa EarlyStopping pozwala na wcześniejsze zatrzymanie procesu trenowania sieci, gdy obserwowna metryka zacznie się pogarszać.

In [25]:
model = build_model()
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2, batch_size=32)

Train on 48000 samples, validate on 12000 samples
Epoch 1/30
48000/48000 [==============================] - 8s 162us/sample - loss: 0.3218 - accuracy: 0.9068 - val_loss: 0.1670 - val_accuracy: 0.9512
Epoch 2/30
48000/48000 [==============================] - 6s 120us/sample - loss: 0.1639 - accuracy: 0.9529 - val_loss: 0.1273 - val_accuracy: 0.9647
Epoch 3/30
48000/48000 [==============================] - 6s 118us/sample - loss: 0.1306 - accuracy: 0.9630 - val_loss: 0.1111 - val_accuracy: 0.9697
Epoch 4/30
48000/48000 [==============================] - 6s 126us/sample - loss: 0.1141 - accuracy: 0.9683 - val_loss: 0.1126 - val_accuracy: 0.9705
Epoch 5/30
48000/48000 [==============================] - 6s 134us/sample - loss: 0.1027 - accuracy: 0.9722 - val_loss: 0.1021 - val_accuracy: 0.9733
Epoch 6/30
48000/48000 [==============================] - 6s 118us/sample - loss: 0.0967 - accuracy: 0.9739 - val_loss: 0.1138 - val_accuracy: 0.9728
Epoch 7/30
48000/48000 [==========================

In [26]:
metrics = pd.DataFrame(history.history)
metrics.head()

,loss,accuracy,val_loss,val_accuracy
0,0.321848,0.906771,0.167017,0.951167
1,0.163875,0.952854,0.127349,0.964667
2,0.130572,0.963000,0.111096,0.969667
3,0.114134,0.968313,0.112647,0.970500
4,0.102703,0.972250,0.102103,0.973333


In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=metrics['loss'], name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['val_loss'], name='val_loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['accuracy'], name='accuracy'), row=2, col=1)
fig.add_trace(go.Scatter(y=metrics['val_accuracy'], name='val_accuracy'), row=2, col=1)
fig.update_layout(width=800)

In [35]:
from tensorflow.keras.callbacks import EarlyStopping

# domyślnie mode='auto'
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

model = build_model()
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2, batch_size=32, callbacks=[es])

Train on 48000 samples, validate on 12000 samples
Epoch 1/30
48000/48000 [==============================] - 6s 126us/sample - loss: 0.3182 - accuracy: 0.9071 - val_loss: 0.1588 - val_accuracy: 0.9543
Epoch 2/30
48000/48000 [==============================] - 6s 118us/sample - loss: 0.1661 - accuracy: 0.9520 - val_loss: 0.1247 - val_accuracy: 0.9652
Epoch 3/30
48000/48000 [==============================] - 6s 118us/sample - loss: 0.1322 - accuracy: 0.9628 - val_loss: 0.1147 - val_accuracy: 0.9674
Epoch 4/30
48000/48000 [==============================] - 7s 145us/sample - loss: 0.1132 - accuracy: 0.9678 - val_loss: 0.1170 - val_accuracy: 0.9679
Epoch 5/30
48000/48000 [==============================] - 6s 118us/sample - loss: 0.1022 - accuracy: 0.9716 - val_loss: 0.1112 - val_accuracy: 0.9717
Epoch 6/30
48000/48000 [==============================] - 6s 119us/sample - loss: 0.0945 - accuracy: 0.9741 - val_loss: 0.1118 - val_accuracy: 0.9723
Epoch 7/30
48000/48000 [==========================

In [ ]:
metrics = pd.DataFrame(history.history)
metrics.head()

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=metrics['loss'], name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['val_loss'], name='val_loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['accuracy'], name='accuracy'), row=2, col=1)
fig.add_trace(go.Scatter(y=metrics['val_accuracy'], name='val_accuracy'), row=2, col=1)
fig.update_layout(width=800)

Tensorboard

In [36]:
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime as dt

!rm -rf ./logs
log_dir = 'logs/fit/' + dt.now().strftime('%Y%m%d-%H%M%S')
tensorboard = TensorBoard(log_dir=log_dir)

In [37]:
model = build_model()
history = model.fit(X_train, y_train, epochs=3, validation_split=0.2, batch_size=32, callbacks=[tensorboard])

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 7s 136us/sample - loss: 0.3179 - accuracy: 0.9066 - val_loss: 0.1616 - val_accuracy: 0.9531
Epoch 2/3
48000/48000 [==============================] - 6s 118us/sample - loss: 0.1644 - accuracy: 0.9514 - val_loss: 0.1246 - val_accuracy: 0.9660
Epoch 3/3
48000/48000 [==============================] - 6s 120us/sample - loss: 0.1315 - accuracy: 0.9621 - val_loss: 0.1120 - val_accuracy: 0.9678


In [38]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit